<a href="https://colab.research.google.com/github/alan-medina-gomez-lic/Evidencia1_Ago2025/blob/main/Evidencia1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from datetime import datetime, timedelta

contador_clientes = 1
contador_salas = 1
contador_folios = 1

TURNOS_POSIBLES = ['MATUTINO', 'VESPERTINO', 'NOCTURNO']

CLIENTES_DF = pd.DataFrame(columns=['Nombre', 'Apellidos'])
SALAS_DF = pd.DataFrame(columns=['Nombre', 'Cupo'])
RESERVACIONES_DF = pd.DataFrame(columns=['Cliente_ID', 'Sala_ID', 'Fecha', 'Turno', 'Evento'])

def registrar_sala(nombre, cupo):
    """Registra una nueva sala con clave única generada automáticamente."""
    global contador_salas, SALAS_DF

    try:
        cupo = int(cupo)
        if cupo <= 0:
            print("Error: El cupo debe ser un número entero positivo.")
            return False
    except ValueError:
        print("Error: El cupo debe ser un número válido.")
        return False

    nueva_clave = str(contador_salas)

    nueva_sala_data = pd.DataFrame({'Nombre': [nombre.upper()], 'Cupo': [cupo]}, index=[nueva_clave])

    SALAS_DF = pd.concat([SALAS_DF, nueva_sala_data])
    contador_salas += 1

    print(f"Sala '{nombre.upper()}' registrada con éxito. Clave: {nueva_clave}, Cupo: {cupo}.")
    return True

def registrar_cliente(nombre, apellidos):
    """Registra un nuevo cliente con clave única generada automáticamente."""
    global contador_clientes, CLIENTES_DF

    if not nombre.strip() or not apellidos.strip():
        print("Error: El nombre y los apellidos del cliente no pueden estar vacíos.")
        return False

    nueva_clave = f'C{contador_clientes}'

    nuevo_cliente_data = pd.DataFrame({'Nombre': [nombre.strip()], 'Apellidos': [apellidos.strip()]}, index=[nueva_clave])

    CLIENTES_DF = pd.concat([CLIENTES_DF, nuevo_cliente_data])
    contador_clientes += 1

    print(f"Cliente '{nombre.strip()} {apellidos.strip()}' registrado con éxito. Clave: {nueva_clave}.")
    return True

def validar_fecha(fecha_str):
    """Valida el formato DD/MM/AAAA."""
    try:
        fecha_reserva = datetime.strptime(fecha_str, '%d/%m/%Y').date()
    except ValueError:
        print("Error: Formato de fecha incorrecto. Use el formato DD/MM/AAAA (ej. 20/10/2000).")
        return None
    return fecha_reserva

def seleccionar_cliente():
    """Muestra clientes ordenados alfabéticamente por apellidos y valida la clave."""

    if CLIENTES_DF.empty:
        print("No hay clientes registrados. Debe registrar un cliente primero.")
        return None

    while True:
        clientes_ordenados = CLIENTES_DF.sort_values(by=['Apellidos', 'Nombre'])

        print("\n--- CLIENTES REGISTRADOS ---")
        print(clientes_ordenados)
        print("----------------------------")

        clave_ingresada = input("Ingrese la CLAVE del cliente (ej. C1) o 'C' para cancelar: ").strip().upper()

        if clave_ingresada == 'C':
            print("Operación de reserva cancelada.")
            return None

        if clave_ingresada in CLIENTES_DF.index:
            return clave_ingresada
        else:
            print(f"La clave '{clave_ingresada}' no existe. Intente de nuevo.")

def registrar_reservacion():
    """Registra una reservación completa con todas las validaciones."""
    global RESERVACIONES_DF, contador_folios, SALAS_DF, CLIENTES_DF

    print("\n\n*** REGISTRAR NUEVA RESERVACIÓN ***")

    if CLIENTES_DF.empty or SALAS_DF.empty:
        print("Error: No hay clientes o salas registradas.")
        return

    cliente_id = seleccionar_cliente()
    if cliente_id is None:
        return

    while True:
        fecha_str = input("Ingrese la FECHA de la reserva (DD/MM/AAAA): ").strip()
        fecha_reserva = validar_fecha(fecha_str)

        if fecha_reserva:
            fecha_actual_sistema = datetime.now().date()
            fecha_minima_reserva = fecha_actual_sistema + timedelta(days=2)

            if fecha_reserva >= fecha_minima_reserva:
                fecha_almacenar = fecha_reserva.strftime('%Y-%m-%d')
                break
            else:
                print(f"La fecha por reservar debe ser, por lo menos, dos días posteriores a la fecha actual del sistema. (Fecha mínima: {fecha_minima_reserva.strftime('%d/%m/%Y')}).")


    query_str = f"Fecha == '{fecha_almacenar}'"
    reservas_del_dia = RESERVACIONES_DF.query(query_str, inplace=False)

    slots_ocupados = reservas_del_dia[['Sala_ID', 'Turno']].values.tolist()

    slots_disponibles = []
    print(f"\n--- DISPONIBILIDAD PARA EL DÍA {fecha_reserva.strftime('%d/%m/%Y')} ---")
    print("CLAVE | NOMBRE | CUPO | TURNO(S) DISPONIBLE(S)")
    print("-------------------------------------------------")

    for sala_id, sala_info in SALAS_DF.iterrows():
        turnos_disponibles_sala = []

        for turno in TURNOS_POSIBLES:
            slot = (sala_id, turno)
            if slot not in slots_ocupados:
                turnos_disponibles_sala.append(turno)
                slots_disponibles.append(slot)

        if turnos_disponibles_sala:
            print(f"{sala_id:<5} | {sala_info['Nombre']:<6} | {sala_info['Cupo']:<4} | {', '.join(turnos_disponibles_sala)}")
        else:
            print(f"{sala_id:<5} | {sala_info['Nombre']:<6} | {sala_info['Cupo']:<4} | (COMPLETO)")

    if not slots_disponibles:
        print("\nLo sentimos, no hay salas disponibles para esa fecha.")
        return

    slot_elegido = None
    while slot_elegido is None:
        sala_elegida = input("\nIngrese la CLAVE de la sala a reservar: ").strip()

        if sala_elegida not in SALAS_DF.index:
            print(f"Error: La clave de sala '{sala_elegida}' no existe. Debe ser una clave numérica del listado (ej. 1, 2, 3).")
            continue

        turno_elegido = input("Ingrese el TURNO (MATUTINO/VESPERTINO/NOCTURNO): ").strip().upper()

        slot_intento = (sala_elegida, turno_elegido)

        if slot_intento in slots_disponibles:
            slot_elegido = slot_intento
        else:
            print("Combinación no disponible o inválida. Revise el listado.")

    sala_id_final, turno_final = slot_elegido

    while True:
        nombre_evento = input("Ingrese el NOMBRE del evento: ").strip().upper()
        if nombre_evento:
            break
        print("El nombre del evento no puede estar vacío.")

    folio_unico = f'F{contador_folios}'

    nueva_reserva_data = pd.DataFrame({'Cliente_ID': [cliente_id], 'Sala_ID': [sala_id_final], 'Fecha': [fecha_almacenar], 'Turno': [turno_final], 'Evento': [nombre_evento]}, index=[folio_unico])

    RESERVACIONES_DF = pd.concat([RESERVACIONES_DF, nueva_reserva_data])
    contador_folios += 1

    print(f"\nRESERVACIÓN REGISTRADA CON ÉXITO. FOLIO: {folio_unico}")

def editar_reservacion():
    """Edita el nombre del evento de una reservación existente."""
    global RESERVACIONES_DF

    if RESERVACIONES_DF.empty:
        print("No hay reservaciones registradas para editar.")
        return

    print("\n\n*** EDITAR NOMBRE DE EVENTO ***")

    while True:
        fecha_inicio_str = input("Ingrese FECHA DE INICIO del rango (DD/MM/AAAA): ").strip()
        fecha_fin_str = input("Ingrese FECHA DE FIN del rango (DD/MM/AAAA): ").strip()

        try:
            fecha_inicio = datetime.strptime(fecha_inicio_str, '%d/%m/%Y').strftime('%Y-%m-%d')
            fecha_fin = datetime.strptime(fecha_fin_str, '%d/%m/%Y').strftime('%Y-%m-%d')
            if fecha_inicio <= fecha_fin:
                break
            else:
                 print("La fecha de inicio debe ser anterior o igual a la fecha de fin.")
        except ValueError:
            print("Formato de fecha incorrecto. Use DD/MM/AAAA.")

    query_str_rango = f"Fecha >= '{fecha_inicio}' and Fecha <= '{fecha_fin}'"
    eventos_en_rango = RESERVACIONES_DF.query(query_str_rango, inplace=False)

    if eventos_en_rango.empty:
        print(f"No se encontraron eventos entre {fecha_inicio_str} y {fecha_fin_str}.")
        return

    df_mostrar = eventos_en_rango[['Evento', 'Fecha']].copy()
    df_mostrar.index.name = 'FOLIO'

    while True:
        print("\n--- EVENTOS ENCONTRADOS ---")
        print(df_mostrar)
        print("----------------------------")

        folio_a_modificar = input("Ingrese el FOLIO del evento a modificar (ej. F1) o 'C' para cancelar: ").strip().upper()

        if folio_a_modificar == 'C':
            print("Edición cancelada.")
            return

        if folio_a_modificar in eventos_en_rango.index:

            while True:
                nuevo_evento = input(f"Ingrese el nuevo nombre para el evento '{folio_a_modificar}': ").strip().upper()
                if nuevo_evento:
                    break
                print("El nombre del evento no puede estar vacío.")

            RESERVACIONES_DF.loc[folio_a_modificar, 'Evento'] = nuevo_evento
            print(f"Evento con FOLIO {folio_a_modificar} modificado a: '{nuevo_evento}'.")
            return

        else:
            print(f"Folio '{folio_a_modificar}' no encontrado en el rango de fechas. Intente de nuevo.")

def consultar_reservaciones():
    """Genera el reporte tabular para una fecha específica, con formato limpio y alineado."""
    global CLIENTES_DF, SALAS_DF, RESERVACIONES_DF

    print("\n\n*** CONSULTAR RESERVACIONES ***")

    if RESERVACIONES_DF.empty:
        print("No hay reservaciones registradas para consultar.")
        return

    while True:
        fecha_str = input("Ingrese la FECHA a consultar (DD/MM/AAAA): ").strip()
        try:
            fecha_consulta = datetime.strptime(fecha_str, '%d/%m/%Y').strftime('%Y-%m-%d')
            break
        except ValueError:
            print("Formato de fecha incorrecto. Use DD/MM/AAAA.")

    query_str_consulta = f"Fecha == '{fecha_consulta}'"
    reservas_del_dia = RESERVACIONES_DF.query(query_str_consulta, inplace=False)

    if reservas_del_dia.empty:
        print(f"No hay reservaciones para la fecha {fecha_str}.")
        return

    reservas_del_dia = reservas_del_dia.copy()

    reservas_del_dia.loc[:, 'CLIENTE_NOMBRE'] = reservas_del_dia['Cliente_ID'].apply(
        lambda x: CLIENTES_DF.loc[x]['Nombre']
    )

    reservas_del_dia.loc[:, 'SALA_CLAVE'] = reservas_del_dia['Sala_ID']

    reporte = reservas_del_dia[['SALA_CLAVE', 'CLIENTE_NOMBRE', 'Evento', 'Turno']].copy()

    fecha_reporte = datetime.strptime(fecha_consulta, '%Y-%m-%d').strftime('%d/%m/%Y')

    ANCHO_TOTAL = 80

    print("\n" + "*" * ANCHO_TOTAL)
    texto_encabezado = f"REPORTE DE RESERVACIONES PARA EL DÍA {fecha_reporte}"
    print(f"**{texto_encabezado:^76}**")
    print("*" * ANCHO_TOTAL)

    print(f"{'SALA':<5} {'CLIENTE':<12} {'EVENTO':<40} {'TURNO':<20}")
    print("*" * ANCHO_TOTAL)

    for index, row in reporte.iterrows():
        sala = str(row['SALA_CLAVE'])[:4]
        cliente = str(row['CLIENTE_NOMBRE'])[:11]
        evento = str(row['Evento'])[:39]
        turno = str(row['Turno'])[:19]

        print(f"{sala:<5} {cliente:<12} {evento:<40} {turno:<20}")

    print("*" * 17 + " FIN DEL REPORTE " + "*" * 46 + "\n")

def precargar_datos_ficticios():
    """Carga datos de ejemplo (clientes, salas y reservas) para pruebas inmediatas."""
    global SALAS_DF, CLIENTES_DF, RESERVACIONES_DF
    global contador_salas, contador_clientes, contador_folios

    contador_salas = 1
    contador_clientes = 1
    contador_folios = 1
    SALAS_DF = pd.DataFrame(columns=['Nombre', 'Cupo'])
    CLIENTES_DF = pd.DataFrame(columns=['Nombre', 'Apellidos'])
    RESERVACIONES_DF = pd.DataFrame(columns=['Cliente_ID', 'Sala_ID', 'Fecha', 'Turno', 'Evento'])

    print("\n--- Cargando Datos Ficticios para Pruebas ---")

    registrar_cliente("Gaagle", "Innovacion")
    registrar_cliente("Macrosoft", "Sistemas")
    registrar_cliente("Howlet", "Packer")

    registrar_sala("Sala 5", 15)
    registrar_sala("Sala 2", 8)
    registrar_sala("Sala 6", 20)

    fecha_reporte_ejemplo = '2025-10-20'

    RESERVACIONES_DF = pd.concat([RESERVACIONES_DF, pd.DataFrame({'Cliente_ID': ['C1', 'C2', 'C3'], 'Sala_ID': ['1', '2', '3'], 'Fecha': [fecha_reporte_ejemplo, fecha_reporte_ejemplo, fecha_reporte_ejemplo], 'Turno': ['MATUTINO', 'VESPERTINO', 'MATUTINO'], 'Evento': ['LANZAMIENTO DEL NUEVO LENGUAJE', 'FORO ESTUDIANTIL DE DESARROLLO', 'PRESENTACION DEL NUEVO HARDWARE']}, index=['F1', 'F2', 'F3'])])

    contador_folios = 4

    print(f"--- Precarga completada. Use la fecha 20/10/2025 para ver el reporte de ejemplo. ---")

def menu_principal():
    """Bucle principal de la aplicación que gestiona el menú."""

    precargar_datos_ficticios()

    while True:
        print("\n" + "=" * 40)
        print("    SISTEMA DE RESERVAS COWORKING (FICTICIO)")
        print("=" * 40)
        print("[1] Registrar la reservación de una sala")
        print("[2] Editar el nombre del evento")
        print("[3] Consultar las reservaciones existentes")
        print("[4] Registrar a un nuevo cliente")
        print("[5] Registrar una sala")
        print("[6] Salir")
        print("-" * 40)

        opcion = input("Seleccione una opción: ").strip().upper()

        if opcion == '1':
            registrar_reservacion()
        elif opcion == '2':
            editar_reservacion()
        elif opcion == '3':
            consultar_reservaciones()
        elif opcion == '4':
            print("\n*** REGISTRAR NUEVO CLIENTE ***")
            nombre = input("Nombre del cliente: ").strip()
            apellidos = input("Apellidos del cliente: ").strip()
            registrar_cliente(nombre, apellidos)
        elif opcion == '5':
            print("\n*** REGISTRAR NUEVA SALA ***")
            nombre = input("Nombre de la sala: ").strip()
            cupo = input("Cupo de la sala: ").strip()
            registrar_sala(nombre, cupo)
        elif opcion == '6':

            print("Saliendo del sistema")
            break
        else:
            print("Opción no válida. Intente de nuevo.")

if __name__ == "__main__":
    menu_principal()